In [1]:
!/opt/bin/nvidia-smi
!nvcc --version

Thu Apr 16 14:03:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
  !curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import os

from fastai.vision import *
from fastai.metrics import error_rate

from PIL import Image, ImageChops, ImageEnhance
from pylab import *

In [5]:
!wget 'https://storage.googleapis.com/ml-weights/exp12-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp10-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp9-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp8-export.pkl'
# !wget 'https://storage.googleapis.com/ml-weights/exp7-export.pkl'

--2020-04-16 14:04:00--  https://storage.googleapis.com/ml-weights/exp12-export.pkl
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 2a00:1450:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179065271 (171M) [application/octet-stream]
Saving to: ‘exp12-export.pkl’

exp12-export.pkl    100%[===================>] 170.77M   114MB/s    in 1.5s    

2020-04-16 14:04:02 (114 MB/s) - ‘exp12-export.pkl’ saved [179065271/179065271]



In [7]:
!wget 'https://storage.googleapis.com/sherlock_datasets/CASIA_ELA.zip'

--2020-04-16 14:05:35--  https://storage.googleapis.com/sherlock_datasets/CASIA_ELA.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 2a00:1450:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1388019404 (1.3G) [application/zip]
Saving to: ‘CASIA_ELA.zip’

CASIA_ELA.zip       100%[===================>]   1.29G   168MB/s    in 10s     

2020-04-16 14:05:46 (126 MB/s) - ‘CASIA_ELA.zip’ saved [1388019404/1388019404]



In [0]:
!unzip -o -q CASIA_ELA.zip

In [0]:
exp7_path = Path("/root/.fastai/data/Exp7/")
exp8_path = Path("/root/.fastai/data/Exp8/")
exp9_path = Path("/root/.fastai/data/Exp9/")
exp10_path = Path("/root/.fastai/data/Exp10/")
exp11_path = Path("/root/.fastai/data/Exp11/")
exp12_path = Path("/root/.fastai/data/Exp12/")

In [0]:
learn = load_learner(exp12_path)

In [0]:
def get_predictions(dir):
    path = '/content/data/CASIA_ELA/' + dir + '/'
    folders = os.listdir(path)

    count_fake = 0
    count_real = 0
    conf_list = []

    for folder in folders:
        if os.path.isdir(path + folder):
            new_path = os.path.join(path + folder + '/')
            print(new_path)
            for f in os.listdir(new_path):
                if f !='.DS_Store':
                    img = open_image(new_path + f)
                    pred_class,pred_idx,confidence = learn.predict(img)
                
                    if str(pred_class) == 'Tampered':
                        if 'Tampered' == folder:
                            count_fake += 1
                        conf = float(confidence[1])
                    elif str(pred_class) == 'Authentic':
                        if 'Authentic' == folder:
                            count_real += 1
                        conf = float(confidence[0])
                    conf = conf * 100
                    
                    conf_list.append(conf)
                
    total_fake = len(os.listdir(str(path) + '/Tampered/'))
    total_real = len(os.listdir(str(path) + '/Authentic/'))
    average_conf = sum(conf_list) / len(conf_list)

    accuracy_fake = (count_fake/total_fake) * 100
    accuracy_real = (count_real/total_real) * 100

    results = {
        'Authentic Classification': {
            'Correctness': str(count_real) + '/' + str(total_real),
            'Accuracy': '{:.2f}'.format(accuracy_real)
        },
        'Tampered Classification': {
            'Correctness': str(count_fake) + '/' + str(total_fake),
            'Accuracy': '{:.2f}'.format(accuracy_fake)
        },
        'Average Confidence': '{:.3f}%'.format(average_conf)
    }

    return results

In [28]:
get_predictions('Train')

/content/data/CASIA_ELA/Train/Tampered/
/content/data/CASIA_ELA/Train/Authentic/


{'Authentic Classification': {'Accuracy': '97.03', 'Correctness': '4384/4518'},
 'Average Confidence': '96.188%',
 'Tampered Classification': {'Accuracy': '95.89', 'Correctness': '2961/3088'}}

In [29]:
get_predictions('Valid')

/content/data/CASIA_ELA/Valid/Tampered/
/content/data/CASIA_ELA/Valid/Authentic/


{'Authentic Classification': {'Accuracy': '96.36', 'Correctness': '2382/2472'},
 'Average Confidence': '95.887%',
 'Tampered Classification': {'Accuracy': '94.74', 'Correctness': '1602/1691'}}

In [27]:
get_predictions('Test')

/content/data/CASIA_ELA/Test/Tampered/
/content/data/CASIA_ELA/Test/Authentic/


{'Authentic Classification': {'Accuracy': '94.82', 'Correctness': '476/502'},
 'Average Confidence': '95.852%',
 'Tampered Classification': {'Accuracy': '94.77', 'Correctness': '326/344'}}